In [0]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

In [0]:
%restart_python

This code outputs explority information about what the openMeteo API is capable of outputting. In our actual programs we obviously arent going to output information in printed dataframes but for the purposes of exploring API capabilities and some simple output pruning, results are moved into a core, hourly forecast dataframe, and an 'alerts' dataframe for outputting hours where flight may be optentially hazardous and why.

For the purposes of testing I have limited extracted information to the coordinates of Atlanta, so one could imagine this information informing delays around Hartsfield Jackson, but we could extract weather information about arrival airports and even flight path weather by simply adjusting latitude and longitude passed into the api calls. We could potentially pass in groups of relevants coordinates and analyze weather at all points at the times relevant to the flight.

The full list of hourly (and 15 minute) parameters can be found at https://open-meteo.com/en/docs about halfway down the page, and it includes some potentially helpful variables, like elavation specific cloud cover and wind speed, that may be helpful, as well as specific types of precpitation information.

Obviously, in this test, most of this information is taken reletively low ground, so not all of it will be helpful for in flight weather predictions which should be taken into account in our actual implementation of this API.


In [0]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# ---------- SETUP ----------
# Cache + retry configuration
cache_session = requests_cache.CachedSession(".cache", expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# ---------- API REQUEST ----------
url = "https://api.open-meteo.com/v1/forecast"

# Example location: Hartsfield–Jackson Atlanta International Airport (ATL)
params = {
    "latitude": 33.6407,
    "longitude": -84.4277,
    "hourly": (
        "temperature_2m,"
        "dew_point_2m,"
        "precipitation,"
        "cloud_cover,"
        "visibility,"
        "wind_speed_10m,"
        "wind_direction_10m,"
        "wind_gusts_10m,"
        "pressure_msl"
    ),
    "timezone": "America/New_York",
    "forecast_days": 1,  # only next 24 hours
}

responses = openmeteo.weather_api(url, params=params)
response = responses[0]

print(f"\nAirport coordinates: {response.Latitude()}°, {response.Longitude()}°")
print(f"Elevation: {response.Elevation()} m")
print(f"Timezone offset: {response.UtcOffsetSeconds()} s\n")

# ---------- PARSE HOURLY DATA ----------
hourly = response.Hourly()

# Extract variables (in same order as requested)
variables = [
    "temperature_2m",
    "dew_point_2m",
    "precipitation",
    "cloud_cover",
    "visibility",
    "wind_speed_10m",
    "wind_direction_10m",
    "wind_gusts_10m",
    "pressure_msl"
]

data = {"time": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)}

# Fill in weather variables
for i, var in enumerate(variables):
    data[var] = hourly.Variables(i).ValuesAsNumpy()

# ---------- CREATE DATAFRAME ----------
df = pd.DataFrame(data)

# ---------- DERIVED METRICS ----------
# Compute relative humidity approximation
# RH ≈ 100 × exp((17.625*Td)/(243.04+Td) - (17.625*T)/(243.04+T))
import numpy as np
T = df["temperature_2m"]
Td = df["dew_point_2m"]
df["relative_humidity_%"] = 100 * np.exp((17.625*Td)/(243.04+Td) - (17.625*T)/(243.04+T))

# ---------- OUTPUT ----------
cols_to_show = [
    "time", "temperature_2m", "dew_point_2m", "relative_humidity_%", 
    "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m",
    "visibility", "cloud_cover", "precipitation", "pressure_msl"
]
print("\n=== Hourly Flight-Relevant Weather Data ===")
print(df[cols_to_show].head(10))  # show first 10 hours

# ---------- EXAMPLE ALERTS ----------
# Simple flags for potentially adverse conditions
alerts = df[
    (df["visibility"] < 5000) |         # low visibility < 5 km
    (df["wind_gusts_10m"] > 30) |       # strong gusts > 30 m/s (~67 mph)
    (df["precipitation"] > 2) |         # heavy precipitation > 2 mm/hr
    (df["cloud_cover"] > 90)            # overcast
]

if not alerts.empty:
    print("\n⚠️ Potential Flight Impact Hours Detected:\n")
    print(alerts[["time", "visibility", "wind_gusts_10m", "precipitation", "cloud_cover"]])
else:
    print("\n✅ No significant weather impacts detected in next 24 hours.")